In [ ]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Access the API key
API_KEY = os.getenv("API_KEY")


In [ ]:
import requests as r

# Base URL
url = "https://alphavantageapi.co/timeseries/analytics"

# Parameters
params = {
    "SYMBOLS": "AAPL,NVDA,IBM, DELL, GOOGL",
    "RANGE": "2024-01-01,2024-11-01",
    "INTERVAL": "DAILY",
    "OHLC": "close",
    "CALCULATIONS": "CORRELATION",
    "apikey": "API_KEY" 
}

# Make the request
response = r.get(url, params=params)

# Check the response
print(response.url)  # Verify the constructed URL
print(response.status_code)

In [ ]:
# extract and store in a dataframe
import pandas as pd
analytics_data = response.json()

# convert data to dataframe
corr_data = analytics_data['payload']['RETURNS_CALCULATIONS']['CORRELATION']
corr_df = pd.DataFrame(
    corr_data['correlation'], 
    index=corr_data['index'], 
    columns=corr_data['index']
)

corr_df.head()


In [ ]:
# Fill NaN values by filling in the reverse (symmetry of correlation matrix)
for i in range(len(corr_df.columns)):
    for j in range(i+1, len(corr_df.columns)):
        # If the lower triangle is populated but the upper triangle has NaN, fill it
        if pd.isna(corr_df.iloc[i, j]):
            corr_df.iloc[i, j] = corr_df.iloc[j, i]

corr_df.head()

In [ ]:
# save as .csv
corr_df.to_csv('stocks_correlation.csv', index=True)